<a href="https://colab.research.google.com/github/rafaelghiorzi/S-AES/blob/main/saes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 1 e 2: Implementação S-AES e modo de operação ECB

A Classe SAES serve como um conjunto das funções necessárias para a implementação do algoritmo S-AES ou Simple AES

In [ ]:
import base64

In [ ]:
class SAES:
    def __init__(self):
        self.sbox = [
            [0x9, 0x4, 0xa, 0xb],
            [0xd, 0x1, 0x8, 0x5],
            [0x6, 0x2, 0x0, 0x3],
            [0xc, 0xe, 0xf, 0x7]
        ]

        # Inverse S-Box for decryption
        self.inv_sbox = [
            [0xa, 0x5, 0x9, 0xb],
            [0x1, 0x7, 0x8, 0xf],
            [0x6, 0x0, 0x2, 0x3],
            [0xc, 0x4, 0xd, 0xe]
        ]

        self.rcon1 = 0x80
        self.rcon2 = 0x30

    def print_state(self, state: list, title: str):
        """
        This prints the 16-bit state in a 2x2 matrix format
        """
        print(f"\033[93m{title}:\033[0m")
        print(f"\033[91m┌─────┬─────┐\033[0m")
        print(f"\033[91m│\033[0m {state[0]:02X}  \033[91m│\033[0m {state[1]:02X}  \033[91m│\033[0m")
        print(f"\033[91m├─────┼─────┤\033[0m")
        print(f"\033[91m│\033[0m {state[2]:02X}  \033[91m│\033[0m {state[3]:02X}  \033[91m│\033[0m")
        print(f"\033[91m└─────┴─────┘\033[0m")

    def expand_key(self, key: int):
        """
        This generates the 3 keys for SAES
        Each key is a list of 4 nibbles
        Returns:
            key0, key1, key2: int
        """
        print(f"")
        print(f"\n→ Starting key expansion with key: {key:04X}")

        # First, convert key into 4 nibbles
        w = []
        w.append((key >> 12) & 0xF) # w0
        w.append((key >> 8) & 0xF)  # w1
        w.append((key >> 4) & 0xF)  # w2
        w.append(key & 0xF)         # w3

        # Key 0 is the original key
        key0 = [w[0], w[1], w[2], w[3]]

        # Generate round key 1
        w4 = w[0] ^ self.substitute_nibbles(w[3], self.sbox) ^ (self.rcon1 >> 4)
        w5 = w[1] ^ self.substitute_nibbles(w[0], self.sbox) ^ (self.rcon1 & 0xF)
        w6 = w[2] ^ w4
        w7 = w[3] ^ w5
        key1 = [w4, w5, w6, w7]

        # Generate round key 2
        w8 = w4 ^ self.substitute_nibbles(w7, self.sbox) ^ (self.rcon2 >> 4)
        w9 = w5 ^ self.substitute_nibbles(w6, self.sbox) ^ (self.rcon2 & 0xF)
        w10 = w6 ^ w8
        w11 = w7 ^ w9
        key2 = [w8, w9, w10, w11]

        print(f"← Key expansion complete")
        return key0, key1, key2

    def add_round_key(self, state: list, key: list) -> list:
        """
        This XORs the state with the key
        XOR nibble by nibble

        Returns:
            new_state: list
        """
        print(f"\n→ Adding round key:")

        new_state = []
        for i in range(4):
            # iterating through each nibble
            new_nibble = state[i] ^ key[i]
            new_state.append(new_nibble)

        print(f"← Add Round Key complete")
        return new_state

    def substitute_nibbles(self, state: list, sbox: list[list]) -> list:
        """
        This applies substitution using the provided S-Box
        on the entire state, nibble by nibble.

        Returns:
            new_state: list or int (if single nibble)
        """
        print(f"\n→ Substituting nibbles:")

        if isinstance(state, int):
            nibble = state
            row = (nibble >> 2) & 0x3
            col = nibble & 0x3
            new_nibble = sbox[row][col]
            print(f"← Substitution complete")
            return new_nibble

        new_state = []
        for nibble in state:
            row = (nibble >> 2) & 0x3 # upper 2 bits for row
            col = nibble & 0x3 # lower 2 bits for column
            new_nibble = sbox[row][col]
            new_state.append(new_nibble)

        print(f"← Substitution complete")
        return new_state

    def shift_rows(self, state: list) -> list:
        """
        This shifts the rows of the state
        the bottom row is shifted

        Returns:
            new_state: list
        """

        print(f"\n→ Shifting rows:")

        new_state = [state[0], state[1], state[3], state[2]]

        print(f"← Shift Rows complete")
        return new_state

    def gf_multiply(self, a: int, b: int) -> int:
        """
        This performs Galois Field multiplication of two 4-bit numbers

        Returns:
            result: int
        """
        result = 0
        for i in range(4):      # process 4 bits of b
            if b & 1:           # if current bit of b is 1
                result ^= a     # Add current value of a to result
            high_bit = a & 0x8  # Check if a will overflow
            a <<= 1             # Shift a left by 1 bit
            if high_bit:        # If it overflows
                a ^= 0x13       # Reduce by irreducible polynomial x^4 + x^3 + x^2 + 1
            a &= 0xF            # Keep a within 4 bits
            b >>= 1             # Shift b right by 1 bit
        return result

    def mix_columns(self, state: list, inverse: bool = False) -> list:
        """
        This applies a matrix on the 16-bit state,
        multiplying each column by the matrix

        Args:
            state: list of 4 nibbles representing the state
            inverse: bool, if True, applies the inverse matrix
        Returns:
            new_state: list
        """

        print(f"\n→ Mixing columns:")

        if inverse:
            matrix = [[0x9, 0x2], [0x2, 0x9]]
        else:
            matrix = [[0x1, 0x4], [0x4, 0x1]]
        state_matrix = [[state[0], state[1]], [state[2], state[3]]]

        result = [[0, 0], [0, 0]]
        for i in range(2):
            for j in range(2):
                for k in range(2):
                    result[i][j] ^= self.gf_multiply(matrix[i][k], state_matrix[k][j])
        new_state = [result[0][0], result[0][1], result[1][0], result[1][1]]

        print(f"← Mix Columns complete")
        return new_state

    def encrypt(self, bits: int, key: int) -> int:
        """
        This encrypts a 16-bit integer using SAES

        Args:
            bits: int, 16-bit integer to encrypt
            key: int, 16-bit key for encryption
        Returns:
            encrypted_bits: int, 16-bit encrypted integer
        """

        print("=" * 50)
        print(f"→ Starting simple encryption")
        print(f"bits: {bits:04X}")
        print(f"key: {key:04X}")

        state = [
            (bits >> 12) & 0xF,  # nibble 0
            (bits >> 8) & 0xF,   # nibble 1
            (bits >> 4) & 0xF,   # nibble 2
            bits & 0xF           # nibble 3
        ]

        self.print_state(state, "initial bits state")
        # Expand the keys
        key0, round_key1, round_key2 = self.expand_key(key)

        # Round 0: AddRoundKey
        print(f"\n{'='*20} ROUND 0 {'='*20}")
        state = self.add_round_key(state, key0)
        self.print_state(state, "After Add Round Key")

        # Round 1: SubstituteNibbles with state, ShiftRows, MixColumns, AddRoundKey
        print(f"\n{'='*20} ROUND 1 {'='*20}")
        state = self.substitute_nibbles(state, self.sbox)
        state = self.shift_rows(state)
        state = self.mix_columns(state)
        state = self.add_round_key(state, round_key1)
        self.print_state(state, "State after Round 1")

        # Round 2: SubstituteNibbles with state, ShiftRows, AddRoundKey
        print(f"\n{'='*20} ROUND 2 {'='*20}")
        state = self.substitute_nibbles(state, self.sbox)
        state = self.shift_rows(state)
        state = self.add_round_key(state, round_key2)
        self.print_state(state, "State after Round 2")

        # Final output
        ciphertext = (state[0] << 12) | (state[1] << 8) | (state[2] << 4) | state[3]

        print(f"← Simple encryption complete")
        print("=" * 50)
        return ciphertext

    def encrypt_ecb(self, plaintext: str | int, key: int) -> dict:
        """
        Encrypts bits or a string using SAES in ECB mode.
        It creates 16-bit blocks from the input and encrypts each block separately.

        Args:
            plaintext: str or int, the input to encrypt
            key: int, the 16-bit key for encryption

        Returns:
            bits: blocks of encrypted 16-bit integers
            text: string based on the encrypted bits
            base64_text: base64 encoded string of the encrypted bits
            hex_text: hexadecimal string of the encrypted bits
        """
        print("=" * 50)
        if isinstance(plaintext, int):
            print(f"→ Encrypting string: '{plaintext:04X}'")

            # Split the integer into 16-bit blocks
            blocks = []
            temp = plaintext
            while temp > 0:
                blocks.append(temp & 0xFFFF) # Get last 16 bits
                temp >>= 16
            if not blocks:
                blocks = [0]
            # No padding needed for int
            blocks.reverse() # Reverse to maintain original order

        elif isinstance(plaintext, str):
            print(f"→ Encrypting string: '{plaintext}'")

            # Convert string to bytes
            data = plaintext.encode('utf-8')
            if len(data) % 2 == 0:
                data += b'\x00'
            blocks = []
            for i in range(0, len(data), 2):
                block = int.from_bytes(data[i:i+2], 'big')
                blocks.append(block)

        # Encrypt each block
        encrypted_blocks = [self.encrypt(block, key) for block in blocks]
        # Turn blocks into a single integer
        all_bytes = b''.join(block.to_bytes(2, 'big') for block in encrypted_blocks)

        print(f"← ECB Encryption complete")

        return {
            'bits' :  encrypted_blocks,
            'text' :  all_bytes.decode('utf-8', errors='ignore').rstrip('\x00'),
            'base64_text' : base64.b64encode(all_bytes).decode('utf-8'),
            'hex_text' : ', '.join(f'{block:04X}' for block in encrypted_blocks)
        }

    def decrypt(self, ciphertext: int, key: int) -> int:
        """
        This decrypts a 16-bit integer using SAES

        Args:
            bits: int, 16-bit integer to decrypt
            key: int, 16-bit key for decryption
        Returns:
            decrypted_bits: int, 16-bit decrypted integer
        """

        print("=" * 50)
        print(f"→ Starting simple decryption")
        print(f"bits: {ciphertext:04X}")
        print(f"key: {key:04X}")

        state = [
            (ciphertext >> 12) & 0xF,  # nibble 0
            (ciphertext >> 8) & 0xF,   # nibble 1
            (ciphertext >> 4) & 0xF,   # nibble 2
            ciphertext & 0xF           # nibble 3
        ]

        self.print_state(state, "initial bits state")

        # Expand the keys
        key0, round_key1, round_key2 = self.expand_key(key)

        # Reverse Round 2: AddRoundKey, ShiftRows, SubstituteNibbles
        print(f"\n{'='*20} INVERSE ROUND 2 {'='*20}")
        state = self.add_round_key(state, round_key2)
        state = self.shift_rows(state)
        state = self.substitute_nibbles(state, self.inv_sbox)
        self.print_state(state, "State after inverse Round 2")

        # Reverse Round 1: AddRoundKey, MixColumns, ShiftRows, SubstituteNibbles
        print(f"\n{'='*20} INVERSE ROUND 1 {'='*20}")
        state = self.add_round_key(state, round_key1)
        state = self.mix_columns(state, inverse=True)
        state = self.shift_rows(state)
        state = self.substitute_nibbles(state, self.inv_sbox)
        self.print_state(state, "State after inverse Round 1")

        # Reverse Round 0: AddRoundKey
        print(f"\n{'='*20} INVERSE ROUND 0 {'='*20}")
        state = self.add_round_key(state, key0)
        self.print_state(state, "State after inverse Round 0")

        # convert state back to 16-bit integer
        plaintext = (state[0] << 12) | (state[1] << 8) | (state[2] << 4) | state[3]
        return plaintext

    def decrypt_ecb(self, ciphertext: list, key: int) -> dict:
        """
        Decrypts a list of 16-bit blocks using SAES.
        Args:
            ciphertext: list, the encrypted data to decrypt
            key: int, the 16-bit key for decryption

        Returns:
            decrypted_bits: int or list, the decrypted 16-bit integer(s)
            decrypted_text: str, the decrypted string
        """
        print("=" * 50)
        print(f"→ Decrypting {len(ciphertext)} blocks")

        decrypted_blocks = [self.decrypt(block, key) for block in ciphertext]
        # Convert decrypted blocks to bytes
        all_bytes = b''.join(block.to_bytes(2, 'big') for block in decrypted_blocks)

        # Decode bytes to string, ignoring padding
        decrypted_text = all_bytes.decode('utf-8', errors='ignore')#.rstrip('\x00')
        print(f"← ECB Decryption complete")

        return {
            'decrypted_bits': decrypted_blocks,
            'decrypted_text': decrypted_text
        }

In [ ]:
saes = SAES()
bits = 0b1101001011010011
text = "This is an encrypted message for SAES testing."
key =       0b1101001010011010

## Análise do resultados
Testando os resultados com implementação simples e com strings em modo de operação ECB

In [ ]:
ciphertext_bits = saes.encrypt(bits, key)
decrypted_bits = saes.decrypt(ciphertext_bits, key)

# Print results
print(f"\n{'='*20} RESULTS {'='*20}")
print(f"SAES Encryption and Decryption")
print(f"Key: {key:04X}")
print(f"Bits: {bits:04X}")
print(f"Ciphertext bits: {ciphertext_bits:04X}")
print(f"Decrypted bits: {decrypted_bits:04X}")

→ Starting simple encryption
bits: D2D3
key: D29A
initial bits state:
┌─────┬─────┐
│ 0D  │ 02  │
├─────┼─────┤
│ 0D  │ 03  │
└─────┴─────┘


→ Starting key expansion with key: D29A

→ Substituting nibbles:
← Substitution complete

→ Substituting nibbles:
← Substitution complete

→ Substituting nibbles:
← Substitution complete

→ Substituting nibbles:
← Substitution complete
← Key expansion complete

==================== ROUND 0 ====================

→ Adding round key:
← Add Round Key complete
After Add Round Key:
┌─────┬─────┐
│ 00  │ 00  │
├─────┼─────┤
│ 04  │ 09  │
└─────┴─────┘

==================== ROUND 1 ====================

→ Substituting nibbles:
← Substitution complete

→ Shifting rows:
← Shift Rows complete

→ Mixing columns:
← Mix Columns complete

→ Adding round key:
← Add Round Key complete
State after Round 1:
┌─────┬─────┐
│ 04  │ 04  │
├─────┼─────┤
│ 0C  │ 09  │
└─────┴─────┘

==================== ROUND 2 ====================

→ Substituting nibbles:
← Substitution

In [ ]:
ciphertext = saes.encrypt_ecb(text, key)
decrypted = saes.decrypt_ecb(ciphertext['bits'], key)

In [32]:
# Split because of enormous output
print(f"\n{'='*20} RESULTS {'='*20}")
print(f"SAES ECB Encryption and Decryption Results")
print(f"Key: {key:04X}")
print(f"Text: {text}")
print(f"Ciphertext bits: {ciphertext['bits']}")
print(f"Ciphertext text: {ciphertext['text']}")
print(f"Ciphertext base64: {ciphertext['base64_text']}")
print(f"Ciphertext hex: {ciphertext['hex_text']}")
print("-" * 50)
print(f"Decrypted bits: {decrypted['decrypted_bits']}")
print(f"Decrypted text: {decrypted['decrypted_text']}")
print("=" * 50)


==================== RESULTS ====================
SAES ECB Encryption and Decryption Results
Key: D29A
Text: This is an encrypted message for SAES testing.
Ciphertext bits: [888, 6204, 33360, 64938, 46616, 603, 6140, 39955, 10109, 39073, 62046, 6620, 50561, 51763, 45649, 1022, 60402, 32909, 23971, 58229, 9261, 47560, 46184, 57122]
Ciphertext text: x<P['}^Ł3Q]u$-ȴh"
Ciphertext base64: A3gYPIJQ/aq2GAJbF/ycEyd9mKHyXhncxYHKM7JRA/7r8oCNXaPjdSQtuci0aN8i
Ciphertext hex: 0378, 183C, 8250, FDAA, B618, 025B, 17FC, 9C13, 277D, 98A1, F25E, 19DC, C581, CA33, B251, 03FE, EBF2, 808D, 5DA3, E375, 242D, B9C8, B468, DF22
--------------------------------------------------
Decrypted bits: [21608, 26995, 8297, 29472, 24942, 8293, 28259, 29305, 28788, 25956, 8301, 25971, 29537, 26469, 8294, 28530, 8275, 16709, 21280, 29797, 29556, 26990, 26414, 0]
Decrypted text: This is an encrypted message for SAES testing.  


# Parte 3: Simulação AES usando bibliotecas criptográficas
Utilizando a biblioteca PyCryptoDome para simular o AES real com os seguintes modos de operação:
- ECB
- CBC
- CFB
- OFB
- CTR

Para cada saída, serão usados a mesma mensagem e vetor de inicialização, e serão analisados o tempo de execução e grau de aleatoriedade, apresentando as saídas em base64

In [ ]:
import time
from Crypto import *